In [ ]:
import sys
sys.path.append('../build/Debug/')
from transformations import (quaternion_matrix as quaternion_to_matrix,
                             euler_matrix as euler_to_matrix, 
                             euler_from_quaternion)
from plot import *

import librigid_body_motion_estimation as rigid_body_motion_estimation
%pylab inline
np.set_printoptions(suppress=True)

# Rigid Body Motion Estimation

This notebook presents rigid body motion estimation using geometric algebra. We presents method in both the Euclidean and the conformal model. In the Euclidean model the rigid body transformations are modelled as a rotor and a translation vector and in the conformal model as motors.

The motor estimation in the conformal model presents novel results by estimating the motor through its bivector generator. This is similar to the approaches used in quaternion estimation in 3D.

## Estimating rigid body transformations in the Euclidean model

Consider the 3-dimensional Euclidean space, $\mathbb{E}^3$, with basis $\{ e_1, e_2, e_3 \}$. The corresponding geometric algebra $\mathbb{G}_3$ is of dimension 8 with basis

$$ \{ 1, e_1, e_2, e_3, e_{12}, e_{13}, e_{23}, e_{123} \}.$$

The Euclidean rotors form a subalgebra of $\mathbb{G}_3$ with basis $\{ 1, e_{12}, e_{13}, e_{23}  \}$. Rotors are isomorphic to Quaternions, $\mathbb{H}$.

Given two corresponding, and possibly noisy, point sets the optimal rotor that transform one point set onto the other is found by minimizing the cost function 

$$ E = \frac{1}{2} \| b - R a \tilde{R} + t  \|^2 $$

## Generate data sets

In [ ]:
# Generate rotation matrix
r = euler_to_matrix(0.0,0.0,np.pi/3.0)[:3,:3]
r

In [ ]:
# translation vector
t = np.array([1.0, 0.0, 0.0])
t

In [ ]:
# Generate initial point sets
n_points = 10
points_a = np.random.normal(0.0, 0.8, (n_points,3))
print(points_a)
points_b = np.array([np.dot(r,point.reshape(3,1)).reshape(3) + t for point in points_a])
# Add noise to the rotated point cloud
sigma = 0.09
mu = 0.0
points_b_noisy = points_b + sigma * np.random.randn(n_points,3) + mu
# points_b_noisy = points_b

In [ ]:
plot_xy_projection(points_a, points_b_noisy)

In [ ]:
rtbe = rigid_body_motion_estimation.RotorTranslationVectorBivectorGeneratorEstimation()
b0 = np.array([1.0, 0.0, 0.0]).reshape(3,1)
t00 = np.array([-1.0, 0.0, 0.0]).reshape(3,1)
rt1 = rtbe.run(b0, t00, points_a, points_b_noisy)
rtbe.summary()['brief_report']

In [ ]:
def create_solver_options(use_trust_region_minimizer=True):
    solver_options = {
        'minimizer_type':'TRUST_REGION',
        'trust_region_strategy_type':'LEVENBERG_MARQUARDT',
        'linear_solver_type':'DENSE_QR',
        'max_num_iterations': 10, 
        'num_threads': 4,
        'num_linear_solver_threads':4,
        'parameter_tolerance': 10e-16,
        'function_tolerance': 10e-16,
        'gradient_tolerance': 10e-12,
        'minimizer_progress_to_stdout':False,
        'trust_region_minimizer_iterations_to_dump':[],
        'trust_region_problem_dump_directory':'',
    }
    if not use_trust_region_minimizer:
        solver_options['minimizer_type'] = 'LINE_SEARCH'
        
    return solver_options  

In [ ]:
solver_options = create_solver_options(use_trust_region_minimizer=False)
solver_options['max_num_iterations'] = 100

rte = rigid_body_motion_estimation.RotorTranslationVectorEstimation(solver_options)
r0 = np.array([1.0, 0.0, 0.0, 0.0]).reshape(4,1)
t01 = np.array([-1.0, 0.0, 0.0]).reshape(3,1)
rt2 = rte.run(r0, t01, points_a, points_b_noisy)
print(rte.summary()['full_report'])

In [ ]:
iterations_rtbe = rtbe.summary()['iterations']
iterations_rte = rte.summary()['iterations']
plt.plot([iteration['gradient_max_norm'] for iteration in iterations_rte], label='full rotor')
plt.plot([iteration['gradient_max_norm'] for iteration in iterations_rtbe], label='bivector generator')
plt.legend()
plt.xticks([iteration['iteration'] for iteration in max(iterations_rte, iterations_rtbe)])
plt.xlabel('iteration')
plt.ylabel('gradient max norm')
plt.show()

In [ ]:
rt1

In [ ]:
rt2

In [2]:
gre = rigid_body_motion_estimation.GeneralRotorEstimation()
gr = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).reshape(7,1)
gre_rt = gre.run(gr, points_a, points_b_noisy)
gre.summary()['brief_report']

NameError: name 'rigid_body_motion_estimation' is not defined

In [ ]:
gre_rt

In [ ]:
iterations = gre.summary()['iterations']
plt.plot([iteration['gradient_max_norm'] for iteration in iterations], label='general rotor')
plt.legend()
plt.xticks([iteration['iteration'] for iteration in iterations])
plt.xlabel('iteration')
plt.ylabel('gradient max norm')
plt.show()

In [ ]:
def extract_screw_axis(motor):
    sp = np.sin(np.arccos(motor[0]))
    r = motor[:4]
    t = motor[4:] / -sp
    return r,t

r,t = extract_screw_axis(gre_rt)
t

In [16]:
# R = X(1:3,1:3);
# t = X(1:3,4);
# [n, phi] = RotMat2AxisAngle(R);

# B23 =  n(1);
# B13 = -n(2);
# B12 =  n(3);

# cp = cos(phi/2);
# sp = sin(phi/2);
# t1 = t(1) / 2;
# t2 = t(2) / 2;
# t3 = t(3) / 2;

# M = [ cp, ...
#      -B)12 * sp, ...
#      -B13 * sp, ...
#      -B23 * sp, ...
#      -t1 * cp + (-B12 * t2 - B13 * t3) * sp, ...
#      -t2 * cp + ( B12 * t1 - B23 * t3) * sp, ...
#      -t3 * cp + (-B13 * t1 + B23 * t2) * sp, ...
#       (B12 * t3 - B13 * t2 + B23 * t1) * sp];

%pylab inline
def create_motor(direction, angle, translation):
    d = direction
    a = angle
    t = translation
    B23 =  d[0]
    B13 = -d[1]
    B12 =  d[2]
    cp = cos(angle/2.0)
    sp = sin(angle/2.0)
    t1 = t[0] / 2.0
    t2 = t[1] / 2.0
    t3 = t[2] / 2.0
    M = np.array([cp,
                  -B12 * sp,
                  -B13 * sp,
                  -B23 * sp,
                  -t1 * cp + (-B12 * t2 - B13 * t3) * sp,
                  -t2 * cp + ( B12 * t1 - B23 * t3) * sp, 
                  -t3 * cp + (-B13 * t1 + B23 * t2) * sp, 
                  (B12 * t3 - B13 * t2 + B23 * t1) * sp]).reshape(8,1)
    return M
create_motor([0,0,1], pi/3, [1,1,1])
    

Populating the interactive namespace from numpy and matplotlib


array([[ 0.8660254],
       [-0.5      ],
       [ 0.       ],
       [ 0.       ],
       [-0.6830127],
       [-0.1830127],
       [-0.4330127],
       [ 0.25     ]])

In [ ]:
# M = [0.739516,0.406033,-0.284237,-0.455481,9.03902,16.5613,-18.74,-9.39839]
# theta_half = acos(M(1));

# A = [-M(1)  M(2)  M(3);...
#      -M(2) -M(1)  M(4); ...
#      -M(3) -M(4) -M(1)];
  
# b = [M(5) M(6) M(7)]';

# t = (0.5 * eye(3) * A) \ b;


In [ ]:
m = gr
a = -0.5 * m[0] * np.eye(3) + \
np.array([0.0, -m[1], m[2], 
          m[1], 0.0, -m[3],
         -m[2], m[3], 0.0]).reshape(3,3)

b = np.array([m[4], m[5], m[6]]).reshape(3,1)
print a
np.linalg.solve(a,b)

In [ ]:
gr.ravel(0)

In [ ]:
fig_width_pt = 358.50475  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (sqrt(5.0)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]
params = {
    'axes.labelsize': 9,
    'font.size': 9,
    "font.family": "serif",
    "font.serif": [],
    'legend.fontsize': 9,
    'legend.linewidth': 0.3,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'text.usetex': True,
    'text.latex.preamble':[r'\usepackage{amsmath}', r'\usepackage{bm}'],
    'figure.figsize': fig_size,
    'axes.linewidth': 0.5
}
matplotlib.rcParams.update(params)

In [ ]:
# (points_a, points_b, points_b_noisy) = generate_dataset(10, 0.09, 0.0)

fig, ax = plt.subplots(1)

# ax.set_aspect('equal')
plt.set_cmap('gray')

almost_black = '#262626'
spines_to_remove = ['top', 'right']
for spine in spines_to_remove:
    ax.spines[spine].set_visible(False)
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

spines_to_keep = ['bottom', 'left']
for spine in spines_to_keep:
    ax.spines[spine].set_linewidth(0.5)
    ax.spines[spine].set_color(almost_black)
ax.xaxis.label.set_color(almost_black)
ax.yaxis.label.set_color(almost_black)

c = random.rand(n_points)

x = points_a[:,0]
y = points_a[:,1]
ax.scatter(x, y, c='r',
           edgecolor=almost_black, linewidth=0.15, 
           label=r"$\{a_i\}$")
for i, txt in enumerate(range(n_points)):
    ax.annotate(txt, (x[i],y[i]))
    
x = points_b[:,0]
y = points_b[:,1]
ax.scatter(x,y,c='b',
           edgecolor=almost_black,linewidth=0.15, 
           label=r"$\{a^\prime_i\}$")
for i, txt in enumerate(range(n_points)):
    ax.annotate(txt, (x[i],y[i]))
    
x = points_b_noisy[:,0]
y = points_b_noisy[:,1]
ax.scatter(x,y,c='g',
           edgecolor=almost_black,linewidth=0.15, 
           label=r"$\{b_i\}$")
for i, txt in enumerate(range(n_points)):
    ax.annotate(txt, (x[i],y[i]))
    
ax.title.set_color(almost_black)
ax.axhline(0, color=almost_black, linewidth=0.15)
ax.axvline(0, color=almost_black, linewidth=0.15)

# Remove the line around the legend box, and instead fill it with a light grey
# Also only use one point for the scatterplot legend because the user will 
# get the idea after just one, they don't need three.
light_grey = np.array([float(248)/float(255)]*3)
legend = ax.legend(frameon=True, scatterpoints=1)
rect = legend.get_frame()
rect.set_facecolor(light_grey)
rect.set_linewidth(0.0)

# Change the legend label colors to almost black, too
texts = legend.texts
for t in texts:
    t.set_color(almost_black)

plt.xlabel(r'$\bm{\mathrm{e}}_1$-axis $[m]$')
plt.ylabel(r'$\bm{\mathrm{e}}_2$-axis $[m]$')

save_dir = '/home/lars/Dropbox/sharelatex/2016_phd_thesis_lt/figures'
plt.show()
# fig.savefig(save_dir + '/general_rotor_estimation_xy_proj.pdf')